# Exploratory Data Analysis

In [1]:
import pandas as pd

## Datetime Data Type

### Apakah perbedaan antara `pd.to_datetime()` dengan `astype('datetime64')`?

Method `pd.to_datetime()` lebih fleksibel jika dibandingkan dengan `astype('datetime64')`.
Hal ini dikarenakan penggunaan method `pd.to_datetime()` memungkinkan kita untuk melakukan transformasi bentuk datetime dengan menambahkan parameter-parameter. Contohnya terdapat parameter `format` untuk melakukan formatting urutan tanggal.

In [2]:
pd.to_datetime('30-11-2020', format='%d-%m-%Y') # tanggal, bulan, tahun

Timestamp('2020-11-30 00:00:00')

### Bagaimana cara menghitung selisih atau jeda tanggal?

Selisih tanggal dapat dihitung menggunakan *method* `timedelta()` pada *library* `datetime`.
Berikut dibawah ini adalah ilustrasi penggunaannya:

In [3]:
household = pd.read_csv('data_input/household.csv')
household['purchase_time'] = pd.to_datetime(household['purchase_time'])
household['weekday'] = household['purchase_time'].dt.day_name()
household.head(2)

,receipt_id,receipts_item_id,purchase_time,category,sub_category,format,unit_price,discount,quantity,yearmonth,weekday
0,9622257,32369294,2018-07-22 21:19:00,Rice,Rice,supermarket,128000.0,0,1,2018-07,Sunday
1,9446359,31885876,2018-07-15 16:17:00,Rice,Rice,minimarket,102750.0,0,1,2018-07,Sunday


Tabel di atas adalah sebuah DataFrame `household` yang memuat informasi transaksi kebutuhan rumah tangga. Kolom `purchase_time` berisi informasi tanggal pembelian barang. Jika diketahui durasi pengiriman sebuah barang adalah 2 hari setelah barang dibeli, maka kita dapat menghitung estimasi barang sampai ke tangan pembeli dengan menggunakan *code* berikut:

In [4]:
from datetime import timedelta
household['shipdate_est'] = household['purchase_time'] + timedelta(days=2)
household.head(2)

,receipt_id,receipts_item_id,purchase_time,category,sub_category,format,unit_price,discount,quantity,yearmonth,weekday,shipdate_est
0,9622257,32369294,2018-07-22 21:19:00,Rice,Rice,supermarket,128000.0,0,1,2018-07,Sunday,2018-07-24 21:19:00
1,9446359,31885876,2018-07-15 16:17:00,Rice,Rice,minimarket,102750.0,0,1,2018-07,Sunday,2018-07-17 16:17:00


Apabila ingin lebih spesifik, kita dapat menambahkan parameter waktu lain pada `timedelta()`, mulai dari `weeks` hingga `microseconds` dengan contoh sebagai berikut:

In [5]:
household['shipdate_est_new'] = household['purchase_time'] + timedelta(weeks=0,
                                                                       days=1,
                                                                       hours=2,
                                                                       minutes=3,
                                                                       seconds=4,
                                                                       milliseconds=5,
                                                                       microseconds=6)
household.head(2)

,receipt_id,receipts_item_id,purchase_time,category,sub_category,format,unit_price,discount,quantity,yearmonth,weekday,shipdate_est,shipdate_est_new
0,9622257,32369294,2018-07-22 21:19:00,Rice,Rice,supermarket,128000.0,0,1,2018-07,Sunday,2018-07-24 21:19:00,2018-07-23 23:22:04.005006
1,9446359,31885876,2018-07-15 16:17:00,Rice,Rice,minimarket,102750.0,0,1,2018-07,Sunday,2018-07-17 16:17:00,2018-07-16 18:20:04.005006


### Ketika menggunakan perintah `dt.to_period('W')`, maka secara otomatis rentang *week* yang ditampilkan selalu dimulai dari hari Senin. Apakah ada cara untuk menampilkan rentang *week* yang dimulai dari hari Minggu

Apabila ingin menampilkan rentang *week* yang dimulai dari hari Minggu, maka dapat dituliskan syntax sebagai berikut :

In [6]:
household['purchase_time'].dt.to_period('W-SAT')

0        2018-07-22/2018-07-28
1        2018-07-15/2018-07-21
2        2018-07-15/2018-07-21
3        2018-07-22/2018-07-28
4        2018-07-22/2018-07-28
                 ...          
71995    2017-12-24/2017-12-30
71996    2017-12-10/2017-12-16
71997    2017-12-24/2017-12-30
71998    2017-12-03/2017-12-09
71999    2017-12-17/2017-12-23
Name: purchase_time, Length: 72000, dtype: period[W-SAT]

### Bagaimana cara mengubah nama hari atau bulan kedalam format Bahasa Indonesia?

Cara terbaik untuk mengubah nama hari/bulan ke dalam Bahasa Indonesia adalah dengan mengubah *locale time* terlebih dahulu, kemudian gunakan *method* `strftime()` untuk mengekstrak komponen dari datetimenya.

Berikut adalah [referensi Python strftime](https://strftime.org/) untuk format penulisan strftime.

In [7]:
import locale
locale.setlocale(locale.LC_TIME, "id")

household['purchase_time'].dt.strftime("%A")

0        Minggu
1        Minggu
2        Minggu
3        Selasa
4         Kamis
          ...  
71995      Rabu
71996      Rabu
71997      Rabu
71998     Kamis
71999    Selasa
Name: purchase_time, Length: 72000, dtype: object

## Categorical Data Type

### Bagaimana cara mengetahui level pada category?

Cara mengetahui level pada category dapat menggunakan attribut `cat.categories` atau dapat menggunakan *method* `unique()`.
Berikut adalah contoh implementasinya:

In [8]:
# mengubah tipe data menjadi category
household[['category','sub_category','format','yearmonth','weekday']] = \
household[['category','sub_category','format','yearmonth','weekday']].astype('category')

In [9]:
household['format'].cat.categories

Index(['hypermarket', 'minimarket', 'supermarket'], dtype='object')

In [10]:
household['format'].unique()

['supermarket', 'minimarket', 'hypermarket']
Categories (3, object): ['supermarket', 'minimarket', 'hypermarket']

**Apa yang membedakan `cat.categories` dengan `unique()`**
- `cat.categories` adalah atribut milik tipe data categorical, sehingga tidak dapat digunakan selain tipe data categorical. 
- `unique()` adalah *method* yang dimiliki oleh `pandas` sehingga dapat digunakan secara fleksible untuk tipe data lainnya.

## Contingency Table

### Apakah parameter `aggfunc` pada crosstab dapat menampilkan lebih dari satu fungsi aggregasi?

Parameter `aggfunc` dapat menampilkan lebih dari satu fungsi aggregasi. Berikut adalah contoh implementasi menampilkan tabulasi silang pada data `household` menggunakan aggregasi `mean` dan `sum`:

In [11]:
pd.crosstab(
    index=household['category'],
    columns=household['format'],
    values=household['unit_price'],
    aggfunc='mean'
)

format,hypermarket,minimarket,supermarket
category,,,
Fabric Care,19328.141915,17757.135774,17847.557137
Rice,71205.458458,67135.569554,74921.182150
Sugar/Flavored Syrup,13539.915728,12352.135470,13071.112361


### Apa kegunaan parameter `margins=True` pada method `pd.crosstab`?

Parameter `margins=True` pada crosstab digunakan untuk menjumlahkan hasil cross tabulasi, sehingga ada kolom tambahan dengan nama kolom `"All"` yang berisi total nilai.

In [12]:
household_crosstab = \
pd.crosstab(
    index=household['category'],
    columns=household['format'],
    values=household['unit_price'],
    aggfunc='mean',
    margins=True
)
household_crosstab

format,hypermarket,minimarket,supermarket,All
category,,,,
Fabric Care,19328.141915,17757.135774,17847.557137,17893.793214
Rice,71205.458458,67135.569554,74921.182150,70013.146313
Sugar/Flavored Syrup,13539.915728,12352.135470,13071.112361,12645.066024
All,27079.468095,23460.177971,27457.145330,24830.776334


### Apakah kita dapat melakukan `sort_values()` pada kolom `All` saat parameter `margins=True`?

Bisa, kita dapat melakukan pengurutan seperti biasa dengan menganggap kolom `All` seperti kolom lainnya. Berikut adalah contohnya, `axis=0` digunakan untuk memastikan bahwa **baris** diurutkan berdasarkan kolom `All`:

In [13]:
household_crosstab.sort_values(by='All', axis=0)

format,hypermarket,minimarket,supermarket,All
category,,,,
Sugar/Flavored Syrup,13539.915728,12352.135470,13071.112361,12645.066024
Fabric Care,19328.141915,17757.135774,17847.557137,17893.793214
All,27079.468095,23460.177971,27457.145330,24830.776334
Rice,71205.458458,67135.569554,74921.182150,70013.146313


Sedangkan apabila kita menggunakan `axis=1`, **kolom** akan diurutkan berdasarkan baris `All`:

In [14]:
household_crosstab.sort_values(by='All', axis=1)

format,minimarket,All,hypermarket,supermarket
category,,,,
Fabric Care,17757.135774,17893.793214,19328.141915,17847.557137
Rice,67135.569554,70013.146313,71205.458458,74921.182150
Sugar/Flavored Syrup,12352.135470,12645.066024,13539.915728,13071.112361
All,23460.177971,24830.776334,27079.468095,27457.145330


## Missing Values

### Bagaimana menampilkan data yang memiliki missing values saja?

Objek `household2` berikut adalah contoh DataFrame yang memiliki beberapa *missing values* didalamnya. 

In [15]:
household = pd.read_csv("data_input/household.csv", index_col=1, parse_dates=['purchase_time'])
household.drop(['receipt_id', 'yearmonth', 'sub_category'], axis=1, inplace=True)
household['weekday'] = household['purchase_time'].dt.day_name()
import math
x=[i for i in range(32000000, 32000005)]
x.insert(2,32030785)
household2 = household.head(6).copy()
household2 = household2.reindex(x)
household2 = pd.concat([household2, household.head(14)])
household2.loc[31885876, "weekday"] = math.nan

In [16]:
household2

,purchase_time,category,format,unit_price,discount,quantity,weekday
receipts_item_id,,,,,,,
32000000,NaT,NaN,NaN,NaN,NaN,NaN,NaN
32000001,NaT,NaN,NaN,NaN,NaN,NaN,NaN
32030785,2018-07-17 18:05:00,Rice,minimarket,63500.0,0.0,1.0,Tuesday
32000002,NaT,NaN,NaN,NaN,NaN,NaN,NaN
32000003,NaT,NaN,NaN,NaN,NaN,NaN,NaN
32000004,NaT,NaN,NaN,NaN,NaN,NaN,NaN
32369294,2018-07-22 21:19:00,Rice,supermarket,128000.0,0.0,1.0,Sunday
31885876,2018-07-15 16:17:00,Rice,minimarket,102750.0,0.0,1.0,NaN
31930241,2018-07-15 12:12:00,Rice,supermarket,64000.0,0.0,3.0,Sunday


Berikut adalah kode yang dapat digunakan untuk menampilkan data yang memiliki *missing values* saja.
- Method `.isna()` digunakan untuk melakukan pengecekan *missing values*
- Method `.any(axis=1)` mengecek apakah terdapat *missing values* pada **kolom**.

In [17]:
household2[household2.isna().any(axis=1)]

,purchase_time,category,format,unit_price,discount,quantity,weekday
receipts_item_id,,,,,,,
32000000,NaT,NaN,NaN,NaN,NaN,NaN,NaN
32000001,NaT,NaN,NaN,NaN,NaN,NaN,NaN
32000002,NaT,NaN,NaN,NaN,NaN,NaN,NaN
32000003,NaT,NaN,NaN,NaN,NaN,NaN,NaN
32000004,NaT,NaN,NaN,NaN,NaN,NaN,NaN
31885876,2018-07-15 16:17:00,Rice,minimarket,102750.0,0.0,1.0,NaN


### Bagaimana cara menangani *missing values* pada data?

Terdapat berbagai cara menangani data yang mengandung *missing value*, yang paling umum adalah *deletion*, *full analysis*, dan *imputation*.

**a. Deletion** adalah **membuang variabel/kolom** pada data yang memiliki jumlah *missing value* (NA) melebihi 50% dari jumlah observasi. Hal ini menganggap bahwa variabel tersebut tidak banyak memberikan informasi pada data. Ketika membuang variabel perlu memperhatikan business case dari data tersebut, apakah variabel yang dibuang akan menghilangkan informasi yang cukup signifikan atau tidak? Apakah ketika variabel tersebut dibuang ada informasi yang berkurang dari data tersebut atau tidak?

**b. Full analysis** adalah **membuang observasi/baris** yang mengandung *missing value*. Cara ini dilakukan jika jumlah observasi yang mengandung *missing value* tidak melebihi 5% dari total observasi data.

**c. Imputation** adalah **mengisi *missing value*** dengan suatu nilai tertentu. Cara ini dilakukan apabila jumlah missing value pada data cukup banyak (melebihi 5% jumlah observasi). Imputation umum dilakukan berdasarkan business knowledge dari variabel tersebut. Misalkan variabel jumlah pengunjung per jam pada sebuah restoran, seharusnya terdapat jumlah visitor yang bernilai 0 pada jam-jam tertentu karena pasti terdapat kemungkinan pada jam-jam tertentu tidak terdapat pengunjung. Apabila secara business knowledge seharusnya observasi tersebut memiliki suatu nilai, kita bisa melakukan imputation dengan menggunakan pusat datanya seperti mean/median untuk variabel numerik dan modus untuk variabel kategorik.

In [18]:
# fillna(0)


In [19]:
# imputasi menggunakan pusat data
